## Cadences Network Visualization with Louvain Communities



In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


In [2]:
# Example using batch to count the cadence types from multiple pieces:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0019.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_1.mei',
                    'https://crimproject.org/mei/CRIM_Mass_0019_2.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_3.mei'
                    ,'https://crimproject.org/mei/CRIM_Mass_0019_4.mei',
                    'https://crimproject.org/mei/CRIM_Mass_0019_5.mei'])
list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
combined_df['all_features'] = combined_df['CadType'] + "_" + combined_df['Tone'] + "_" + combined_df['Evaded'].apply(str) + "_" + combined_df['Beat'].apply(str)
combined_df['some_features'] = combined_df['CadType'] + "_" + combined_df['Tone']
combined_df.copy
combined_df["shifted_all_feat"] = combined_df.groupby("Title")['all_features'].shift(-1)

Successfully imported https://crimproject.org/mei/CRIM_Model_0019.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_1.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_2.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_3.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_4.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_5.mei


In [3]:
combined_df.head()

,CadType,Evaded,LeadingTones,Low,RelLow,Tone,RelTone,Measure,Beat,Progress,SinceLast,ToNext,Composer,Title,all_features,some_features,shifted_all_feat
0,NaN,NaN,NaN,G3,P1,G,P1,7,1.0,0.090909,48.0,36.0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,NaN,NaN,Clausula Vera_G_True_3.0
1,Clausula Vera,True,1.0,G3,P1,G,P1,11,3.0,0.159091,36.0,52.0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,Clausula Vera_G_True_3.0,Clausula Vera_G,Authentic_G_True_1.0
2,Authentic,True,1.0,B3,M3,G,P1,18,1.0,0.257576,52.0,16.0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,Authentic_G_True_1.0,Authentic_G,Authentic_G_False_1.0
3,Authentic,False,1.0,G3,P1,G,P1,20,1.0,0.287879,16.0,100.0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,Authentic_G_False_1.0,Authentic_G,Clausula Vera_C_True_3.0
4,Clausula Vera,True,1.0,C4,P4,C,-P5,32,3.0,0.477273,100.0,44.0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,Clausula Vera_C_True_3.0,Clausula Vera_C,Authentic_C_False_1.0


### Network Visualization with Louvain Communities

In [8]:
combined_df = combined_df.dropna(subset=['all_features', 'shifted_all_feat'])
cadence_pairs = list(zip(combined_df.all_features, combined_df.shifted_all_feat))

In [9]:
combined_df['all_features'].nunique()

35

In [10]:
cadence_pairs

[('Clausula Vera_G_True_3.0', 'Authentic_G_True_1.0'),
 ('Authentic_G_True_1.0', 'Authentic_G_False_1.0'),
 ('Authentic_G_False_1.0', 'Clausula Vera_C_True_3.0'),
 ('Clausula Vera_C_True_3.0', 'Authentic_C_False_1.0'),
 ('Authentic_C_False_1.0', 'Authentic_A_False_1.0'),
 ('Clausula Vera_A_False_3.0', 'Authentic_D_False_1.0'),
 ('Authentic_D_False_1.0', 'Altizans Only_D_False_1.0'),
 ('Altizans Only_D_False_1.0', 'Authentic_A_False_3.0'),
 ('Clausula Vera_G_False_1.0', 'Clausula Vera_D_False_3.0'),
 ('Clausula Vera_D_False_3.0', 'Authentic_G_False_1.0'),
 ('Authentic_G_False_1.0', 'Clausula Vera_G_True_3.0'),
 ('Clausula Vera_G_True_3.0', 'Authentic_C_True_4.0'),
 ('Authentic_C_True_4.0', 'Authentic_G_True_2.0'),
 ('Authentic_G_True_2.0', 'Authentic_C_False_3.0'),
 ('Authentic_C_False_3.0', 'Authentic_A_False_3.0'),
 ('Authentic_A_False_3.0', 'Clausula Vera_A_False_3.0'),
 ('Clausula Vera_A_False_3.0', 'Clausula Vera_A_False_3.0'),
 ('Clausula Vera_A_False_3.0', 'Clausula Vera_D_False_

In [11]:
combined_df['all_features'].nunique()

35

In [12]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G
pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="white", font_color="black", directed=True)

In [13]:
# G = nx.Graph()
G = nx.DiGraph()
# G.add_edges_from(unique_pairs)
G.add_edges_from(cadence_pairs)
# G = add_communities(G)
pyvis_graph.from_nx(G)

In [14]:
pyvis_graph.show('MINT.html')


In [41]:
mass_sets = [('CRIM_Mass_0019',
('CRIM_Model_0019.mei',
 'CRIM_Mass_0019_1.mei',
 'CRIM_Mass_0019_2.mei',
 'CRIM_Mass_0019_3.mei',
 'CRIM_Mass_0019_4.mei',
 'CRIM_Mass_0019_5.mei'))]



In [42]:
mass_sets[0][0]

'CRIM_Mass_0019'

In [48]:
for mass in mass_sets:
    corpus_set = []
    csv_name = 'saved_csv/' + mass_sets[0][0] + '_' + 'cadences'
    file_names = mass[1]
    for file in file_names:
        full_file_name = 'https://crimproject.org/mei/' + file
        corpus_set.append(full_file_name)
    
    corpus = CorpusBase(corpus_set)
    list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=True)
    combined_df = pd.concat(list_of_dfs, ignore_index=True)
    combined_df['all_features'] = combined_df['CadType'] + "_" + combined_df['Tone'] + "_" + combined_df['Evaded'].apply(str) + "_" + combined_df['Beat'].apply(str)
    combined_df['some_features'] = combined_df['CadType'] + "_" + combined_df['Tone']
# return(combined_df)
        

Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.


In [49]:
combined_df.to_csv(csv_name)